In [32]:
import torch
import numpy as np
import os
import cv2
from tqdm import tqdm
import glob

class Predictor:
    def __init__(self, imgs, modelName='bestest.pt', modelPath='../data/yoloModels/', outputPath='../data/yoloPredictions/'):
        self.imgs = imgs
        self.outputPath = outputPath
        self.modelName = modelName
        self.model = torch.hub.load('yolov5','custom', path=modelPath+modelName, source='local', _verbose=False)
        self.results = [] # [[[xmin, xmax, ymin, ymax, certainty, classID, className, [centerX, centerY]], ...], ...]
        self.getPredictions()


    def __str__(self):
        return f'{len(self.imgs)} images, {self.name}'

    def getPredictions(self):
        for i, img in enumerate(self.imgs):
            currResult = self.model(img).pandas().xyxy[0].to_numpy()
            self.model(img).save(labels=True, save_dir=f'{os.getcwd()}/{self.outputPath}/{self.modelName[0:-3]}')

            self.results.append([])
            for j, res in enumerate(currResult):
                self.results[-1].append([])
                for k, val in enumerate(res):
                    self.results[-1][-1].append(val)
                
                # self.results[-1][-1].append(i)
                center = self.getCenter(*self.results[-1][-1][:4])
                self.results[-1][-1].append(center)


    def getCenter(self, xmin, xmax, ymin, ymax):
        return np.array([(xmin + xmax)/2, (ymin + ymax)/2]).astype(int)


In [33]:
path = 'data/conveyorImages'
imgs = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.png')]
pred = Predictor(imgs=imgs)

In [14]:
res = [x for x in pred.results if len(x) > 0]
res

[[[632.6787719726562,
   352.8246765136719,
   797.4872436523438,
   510.2147216796875,
   0.9062528610229492,
   2,
   'box',
   1,
   array([492, 653])]]]

In [ ]:
image_folder = 'data/yoloPredictions/right/*'
video_name = 'right_cam_predicted.avi'#save as .avi
#is changeable but maintain same h&w over all  frames
width=1280
height=720 
#this fourcc best compatible for avi
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video=cv2.VideoWriter(video_name,fourcc, 60, (width,height))

for i in tqdm((sorted(glob.glob(image_folder)))):
     x=cv2.imread(i)
     video.write(x)

cv2.destroyAllWindows()
video.release()